In [ ]:
import glob
import numpy as np
import xarray as xr
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from scipy.fft import rfft, irfft, rfftfreq

In [ ]:
import cartopy
cartopy.config['pre_existing_data_dir']='/ihesp/shared/cartopy_features'
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

### WRF output (data credit: Dan Fu)

In [ ]:
#-- Recommend 4 cores with 60GB memory

path = r'/ihesp/shared/fudan1991/CESMdownscale/Thompson/'
#filenames = glob.glob(path + 'cmpr_Thompson_wrfout_downscale*')
filenames = glob.glob(path + 'cmpr_Thompson_wrfout_RAINNC_OLR_downscale*')
filenames.sort()

In [ ]:
filenames = filenames[31:123]

In [ ]:
start_loop = datetime.now()

ds = xr.open_mfdataset(filenames,
                       concat_dim='Time',
                       combine='nested').RAINNC

print('finished cell in: ', datetime.now() - start_loop)

In [ ]:
#-- Create lat, lon arrays
a = ds[0].XLAT.values
lat_vals = [i[0] for i in a]
lon_vals = ds[0][0].XLONG.values

In [ ]:
#time = xr.cftime_range(start="1987-06-01", periods=368, freq="6H", calendar="noleap")
time = xr.cftime_range(start="1987-06-01", periods=2208, freq="1H", calendar="noleap")

In [ ]:
ds = xr.DataArray(
    data=ds.data,
    name="RAINNC",
    dims=["time", "lat", "lon"],
    coords=dict(
                time=(["time"], time),
                lat=(["lat"], lat_vals),
                lon=(["lon"], lon_vals) ))

In [ ]:
ds = ds.sel(lat=slice(15,52), lon=slice(-120,-70))

In [ ]:
%%time
ds.load()

In [ ]:
#-- Post-process cumulative rainfall
start_loop = datetime.now()

ds_fixed = xr.zeros_like(ds)
ds_old = ds.isel(time=0)

for time in np.arange(0, len(ds.time)):
    ds_fixed[time] = ds[time] - ds_old
    ds_old = ds[time]
    
print('finished cell in: ', datetime.now() - start_loop)

In [ ]:
#-- Construct daily mean
daily_mean = ds_fixed.resample(time='1D').mean(dim='time')

In [ ]:
#-- Expand daily mean
n=24
daily_mean_24 = np.repeat(daily_mean.values, n, axis=0)

In [ ]:
%whos

In [ ]:
del ds
del ds_old
del daily_mean

In [ ]:
#-- Compute departure from daily mean
departure_daily_mean = np.subtract(ds_fixed.data, daily_mean_24)

In [ ]:
departure_daily_mean_xr = xr.DataArray(
    data=departure_daily_mean.data,
    dims=["time", "lat", "lon"],
    coords=dict(
                time=(["time"], ds_fixed.time.data),
                lat=(["lat"], ds_fixed.lat.data),
                lon=(["lon"], ds_fixed.lon.data) ))

In [ ]:
#-- Construst a composite day using all departures from daily mean
composite_departure = departure_daily_mean_xr.groupby("time.hour").mean()

In [ ]:
#-- Loop through lat/lon to compute s1 for '24-hour composite day' at each grid point
s_1 = xr.zeros_like(composite_departure)

start_loop = datetime.now()
for lat_idx in np.arange(0, len(composite_departure.lat)):
    for lon_idx in np.arange(0, len(composite_departure.lon)):
        
        y = composite_departure.isel(lat=lat_idx, lon=lon_idx).values
        data_step = 1
        t = np.arange(start=0,stop=len(y),step=data_step)
        n = len(y)
        yf = rfft(y)
        xf = rfftfreq(n,data_step)
        
        yf_abs = np.abs(yf) 
        indices = yf_abs>= yf_abs.max()   # filter out those value under the max value (only take 1st peak)
        yf_clean = indices * yf # noise frequency will be set to 0
        
        new_f_clean = irfft(yf_clean)
        s_1[:,lat_idx,lon_idx] = new_f_clean
        
    print('finished 1 lat loop:', lat_idx, datetime.now() - start_loop)
        
print('finished loop to compute s1:', datetime.now() - start_loop)
        
s_1 = s_1.rename('PRECT_s1')

In [ ]:
s_1

### AMIP-HR 6-hourly PRECT

In [ ]:
#-- HRAMIP 6-hourly precip data - regridded
year = '1981'

path = '/ihesp/user/asblack/regridded/'
f_1 = 'cam_h2_HR_PRECT_'+year+'052700Z_'+year+'070206Z.nc'
f_2 = 'cam_h2_HR_PRECT_'+year+'070212Z_'+year+'080718Z.nc'
f_3 = 'cam_h2_HR_PRECT_'+year+'080800Z_'+year+'091306Z.nc'

ds = xr.open_mfdataset((path+f_1, path+f_2, path+f_3)).PRECT

### 1-hourly PRECT

In [ ]:
#-- HRMIP hourly precip data - regridded
year = '1989'

path = '/ihesp/user/asblack/regridded/'
f_1 = 'cam_h5_HR_PRECT_'+year+'052701Z_'+year+'061406Z.nc'
f_2 = 'cam_h5_HR_PRECT_'+year+'061407Z_'+year+'070212Z.nc'
f_3 = 'cam_h5_HR_PRECT_'+year+'070213Z_'+year+'072018Z.nc'
f_4 = 'cam_h5_HR_PRECT_'+year+'072019Z_'+year+'080800Z.nc'
f_5 = 'cam_h5_HR_PRECT_'+year+'080801Z_'+year+'082606Z.nc'
f_6 = 'cam_h5_HR_PRECT_'+year+'082607Z_'+year+'091312Z.nc'

ds = xr.open_mfdataset((path+f_1, path+f_2, path+f_3,
                        path+f_4, path+f_5, path+f_6)).PRECT

#-- Need to switch to slice by time label instead of location (.sel throws error)
#-- Slice over JJA months
ds = ds.isel(time=slice(119,2327))

In [ ]:
#-- LRMIP hourly precip data - regridded
year = '1988'

path = r'/ihesp/user/asblack/regridded/'
f_1 = 'cam_h5_LR_PRECT_'+year+'052700Z_'+year+'061405Z.nc'
f_2 = 'cam_h5_LR_PRECT_'+year+'061406Z_'+year+'070211Z.nc'
f_3 = 'cam_h5_LR_PRECT_'+year+'070212Z_'+year+'072017Z.nc'
f_4 = 'cam_h5_LR_PRECT_'+year+'072018Z_'+year+'080723Z.nc'
f_5 = 'cam_h5_LR_PRECT_'+year+'080800Z_'+year+'082605Z.nc'
f_6 = 'cam_h5_LR_PRECT_'+year+'082606Z_'+year+'091311Z.nc'

ds = xr.open_mfdataset((path+f_1, path+f_2, path+f_3,
                        path+f_4, path+f_5, path+f_6)).PRECT

#-- Need to switch to slice by time label instead of location (.sel throws error)
#-- Slice over JJA months
ds = ds.isel(time=slice(120,2328))

In [ ]:
#-- Slice over GoM region
global_offset = 360
ds = ds.sel(lat=slice(15,55), lon=slice(global_offset-120,global_offset-70))

In [ ]:
#-- Load into memory
ds.load()

In [ ]:
#-- Construct daily mean
daily_mean = ds.resample(time='1D').mean(dim='time')

In [ ]:
#-- Expand daily mean
n=24
daily_mean_24 = np.repeat(daily_mean.values, n, axis=0)

In [ ]:
#-- Compute departure from daily mean
departure_daily_mean = ds - daily_mean_24

### LRMIP 6-hourly PRECT

In [ ]:
#-- LRMIP 6-hourly (instantaneous) precip data - regridded
year = '1982'

path = '/ihesp/user/asblack/regridded/'
f_1 = 'cam_h2_LR_PRECT_'+year+'052700Z_'+year+'070206Z.nc'
f_2 = 'cam_h2_LR_PRECT_'+year+'070212Z_'+year+'080718Z.nc'
f_3 = 'cam_h2_LR_PRECT_'+year+'080800Z_'+year+'091306Z.nc'

ds = xr.open_mfdataset((path+f_1, path+f_2, path+f_3)).PRECT

In [ ]:
#-- Need to switch to slice by time label instead of location (.sel throws error)
#-- Slice over JJA months
ds = ds.isel(time=slice(20,388))

In [ ]:
#-- Slice over GoM region
global_offset = 360
ds = ds.sel(lat=slice(15,55), lon=slice(global_offset-120,global_offset-70))

In [ ]:
#-- Load into memory
ds.load()

In [ ]:
#-- Construct daily mean
daily_mean = ds.resample(time='1D').mean(dim='time')

In [ ]:
#-- Expand daily mean
n=4
daily_mean_4 = np.repeat(daily_mean.values, n, axis=0)

In [ ]:
#-- Compute departure from daily mean
departure_daily_mean = ds - daily_mean_4

### s1 fourier component of '24-hour composite day'

In [ ]:
#-- Construst a composite day using all departures from daily mean
composite_departure = departure_daily_mean.groupby("time.hour").mean()

In [ ]:
#-- Loop through lat/lon to compute s1 for '24-hour composite day' at each grid point
s_1 = xr.zeros_like(composite_departure)

start_loop = datetime.now()
for lat_idx in np.arange(0, len(composite_departure.lat)):
    for lon_idx in np.arange(0, len(composite_departure.lon)):
        
        y = composite_departure.isel(lat=lat_idx, lon=lon_idx).values
        data_step = 1
        t = np.arange(start=0,stop=len(y),step=data_step)
        n = len(y)
        yf = rfft(y)
        xf = rfftfreq(n,data_step)
        
        yf_abs = np.abs(yf) 
        indices = yf_abs>= yf_abs.max()   # filter out those value under the max value (only take 1st peak)
        yf_clean = indices * yf # noise frequency will be set to 0
        
        new_f_clean = irfft(yf_clean)
        s_1[:,lat_idx,lon_idx] = new_f_clean
        
print('finished loop to compute s1:', datetime.now() - start_loop)
        
s_1 = s_1.rename('PRECT_s1')

In [ ]:
#-- Write to output file - HRMIP 1-hourly precip
ds_PRECT = xr.Dataset(
            data_vars=dict(
                departure_daily_mean=(["time", "lat", "lon"], departure_daily_mean.data),
                daily_mean = (["day", "lat", "lon"], daily_mean.data),
                s_1=(["hour", "lat", "lon"], s_1.data)
            ),
            coords=dict(
                time=(["time"], departure_daily_mean.time.data),
                day=(["day"], daily_mean.time.data),
                hour=(["hour"], s_1.hour.data),
                lat=(["lat"], departure_daily_mean.lat.data),
                lon=(["lon"], departure_daily_mean.lon.data)
            ),
            attrs=dict(description="original file location: /ihesp/archive/HRMIP/B.E.13.B1950TRC5.ne120_t12.cesm-ihesp-1950-2050.shrflx.001/atm/hist/, s_1 computed using scipy fast fourier transform" )
    
)

path = '/ihesp/user/asblack/diurnal_PRECT/'
filename = 'cam_h5_HR_PRECT_s1_'+year+'_JJA.nc'
ds_PRECT.to_netcdf(path + filename)

In [ ]:
#-- Write to output file - LRMIP 1-hourly precip
ds_PRECT = xr.Dataset(
            data_vars=dict(
                departure_daily_mean=(["time", "lat", "lon"], departure_daily_mean.data),
                daily_mean = (["day", "lat", "lon"], daily_mean.data),
                s_1=(["hour", "lat", "lon"], s_1.data)
            ),
            coords=dict(
                time=(["time"], departure_daily_mean.time.data),
                day=(["day"], daily_mean.time.data),
                hour=(["hour"], s_1.hour.data),
                lat=(["lat"], departure_daily_mean.lat.data),
                lon=(["lon"], departure_daily_mean.lon.data)
            ),
            attrs=dict(description="original file location: /scratch/user/asblack/test_CESM13/b.e13.B1950TRC5.ne30_g16.cesm-ihesp-hires1.0.42.hourly.001/run/, s_1 computed using scipy fast fourier transform" )
    
)

path = '/ihesp/user/asblack/diurnal_precip/'
filename = 'cam_h5_LR_PRECT_s1_'+year+'_JJA.nc'
ds_PRECT.to_netcdf(path + filename)

In [ ]:
#-- Write to output file - LRMIP 6-hourly precip
ds_PRECT = xr.Dataset(
            data_vars=dict(
                departure_daily_mean=(["time", "lat", "lon"], departure_daily_mean.data),
                daily_mean = (["day", "lat", "lon"], daily_mean.data),
                s_1=(["hour", "lat", "lon"], s_1.data)
            ),
            coords=dict(
                time=(["time"], departure_daily_mean.time.data),
                day=(["day"], daily_mean.time.data),
                hour=(["hour"], s_1.hour.data),
                lat=(["lat"], departure_daily_mean.lat.data),
                lon=(["lon"], departure_daily_mean.lon.data)
            ),
            attrs=dict(description="original file location: /ihesp/archive/LR/b.e13.B1950TRC5.ne30_g16.ihesp24_1950-2050.002/atm/hist/, s_1 computed using scipy fast fourier transform" )
    
)

path = '/ihesp/user/asblack/diurnal_PRECT/'
filename = 'cam_h2_LR_PRECT_s1_'+year+'_JJA.nc'
ds_PRECT.to_netcdf(path + filename)

In [ ]:
#-- Write to output file - AMIP-HR 6-hourly precip
ds_PRECT = xr.Dataset(
            data_vars=dict(
                departure_daily_mean=(["time", "lat", "lon"], departure_daily_mean.data),
                daily_mean = (["day", "lat", "lon"], daily_mean.data),
                s_1=(["hour", "lat", "lon"], s_1.data)
            ),
            coords=dict(
                time=(["time"], departure_daily_mean.time.data),
                day=(["day"], daily_mean.time.data),
                hour=(["hour"], s_1.hour.data),
                lat=(["lat"], departure_daily_mean.lat.data),
                lon=(["lon"], departure_daily_mean.lon.data)
            ),
            attrs=dict(description="original file location: /ihesp/archive/AMIP/f.e13.FAMIPC5.ne120_ne120_mt12.cesm-ihesp-1950-2050.001/atm/hist/, s_1 computed using scipy fast fourier transform" )
    
)

path = '/ihesp/user/asblack/diurnal_PRECT/'
filename = 'cam_h2_AMIP_HR_PRECT_s1_'+year+'_JJA.nc'
ds_PRECT.to_netcdf(path + filename)

In [ ]:
#-- Write to output file - AMIP-LR 6-hourly precip
ds_PRECT = xr.Dataset(
            data_vars=dict(
                departure_daily_mean=(["time", "lat", "lon"], departure_daily_mean.data),
                daily_mean = (["day", "lat", "lon"], daily_mean.data),
                s_1=(["hour", "lat", "lon"], s_1.data)
            ),
            coords=dict(
                time=(["time"], departure_daily_mean.time.data),
                day=(["day"], daily_mean.time.data),
                hour=(["hour"], s_1.hour.data),
                lat=(["lat"], departure_daily_mean.lat.data),
                lon=(["lon"], departure_daily_mean.lon.data)
            ),
            attrs=dict(description="original file location: /ihesp/archive/AMIP/f.e13.FAMIPC5.ne30_ne30_mt12.ihesp24_1950-2050.001/atm/hist/, s_1 computed using scipy fast fourier transform" )
    
)

path = '/ihesp/user/asblack/diurnal_PRECT/'
filename = 'cam_h2_AMIP_LR_PRECT_s1_'+year+'_JJA.nc'
ds_PRECT.to_netcdf(path + filename)

### Plot check

In [ ]:
year = '1980'

path = '/ihesp/user/asblack/diurnal_precip/'
filename = 'cam_h5_LR_PRECT_s1_'+year+'_JJA.nc'
departure_daily_mean = xr.open_dataset(path + filename).departure_daily_mean
s_1 = xr.open_dataset(path + filename).s_1

In [ ]:
departure_daily_mean

In [ ]:
#-- Construst a composite day using all departures from daily mean
composite_departure = departure_daily_mean.groupby("time.hour").mean()

In [ ]:
#-- Plot side by side
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharey=True)

composite_departure.isel(lat=22, lon=31).plot.line(ax=ax1)
ax1.set_ylabel('composite departure from daily mean')

s_1.isel(lat=22, lon=31).plot.line(ax=ax2)
ax2.set_ylabel('fft s1')

# Show
plt.tight_layout()
plt.show()

In [ ]:
#-- Plot overlapping
f, (ax1) = plt.subplots(figsize=(8, 4))

composite_departure.isel(lat=22,lon=31).plot.line(ax=ax1, label='composite departure')
s_1.isel(lat=22,lon=31).plot.line(ax=ax1, label='fft s1')
ax1.set_ylabel('amplitude')

# Show
ax1.legend()
plt.tight_layout()
plt.show()

### Plot JJA s_1 composites

In [ ]:
# ds_1 = xr.open_dataset('/ihesp/user/asblack/diurnal_precip/cam_h5_HR_PRECT_s1_1980_JJA.nc')
# ds_2 = xr.open_dataset('/ihesp/user/asblack/diurnal_precip/cam_h5_HR_PRECT_s1_1981_JJA.nc')
# ds_3 = xr.open_dataset('/ihesp/user/asblack/diurnal_precip/cam_h5_HR_PRECT_s1_1982_JJA.nc')
# ds_4 = xr.open_dataset('/ihesp/user/asblack/diurnal_precip/cam_h5_HR_PRECT_s1_1983_JJA.nc')
# ds_5 = xr.open_dataset('/ihesp/user/asblack/diurnal_precip/cam_h5_HR_PRECT_s1_1984_JJA.nc')
# ds_6 = xr.open_dataset('/ihesp/user/asblack/diurnal_precip/cam_h5_HR_PRECT_s1_1985_JJA.nc')
# ds_7 = xr.open_dataset('/ihesp/user/asblack/diurnal_precip/cam_h5_HR_PRECT_s1_1986_JJA.nc')
# ds_8 = xr.open_dataset('/ihesp/user/asblack/diurnal_precip/cam_h5_HR_PRECT_s1_1987_JJA.nc')
# ds_9 = xr.open_dataset('/ihesp/user/asblack/diurnal_precip/cam_h5_HR_PRECT_s1_1988_JJA.nc')
# ds_10 = xr.open_dataset('/ihesp/user/asblack/diurnal_precip/cam_h5_HR_PRECT_s1_1989_JJA.nc')

In [ ]:
ds_1 = xr.open_dataset('/ihesp/user/asblack/diurnal_precip/cam_h5_LR_PRECT_s1_1980_JJA.nc')
ds_2 = xr.open_dataset('/ihesp/user/asblack/diurnal_precip/cam_h5_LR_PRECT_s1_1981_JJA.nc')
ds_3 = xr.open_dataset('/ihesp/user/asblack/diurnal_precip/cam_h5_LR_PRECT_s1_1982_JJA.nc')
ds_4 = xr.open_dataset('/ihesp/user/asblack/diurnal_precip/cam_h5_LR_PRECT_s1_1983_JJA.nc')
ds_5 = xr.open_dataset('/ihesp/user/asblack/diurnal_precip/cam_h5_LR_PRECT_s1_1984_JJA.nc')
ds_6 = xr.open_dataset('/ihesp/user/asblack/diurnal_precip/cam_h5_LR_PRECT_s1_1985_JJA.nc')
ds_7 = xr.open_dataset('/ihesp/user/asblack/diurnal_precip/cam_h5_LR_PRECT_s1_1986_JJA.nc')
ds_8 = xr.open_dataset('/ihesp/user/asblack/diurnal_precip/cam_h5_LR_PRECT_s1_1987_JJA.nc')
ds_9 = xr.open_dataset('/ihesp/user/asblack/diurnal_precip/cam_h5_LR_PRECT_s1_1988_JJA.nc')

composite_diurnal_peak_amplitude = (ds_1.s_1.max(dim='hour') + ds_2.s_1.max(dim='hour') + ds_3.s_1.max(dim='hour')
                                   +ds_4.s_1.max(dim='hour') + ds_5.s_1.max(dim='hour') + ds_6.s_1.max(dim='hour')
                                   +ds_7.s_1.max(dim='hour') + ds_8.s_1.max(dim='hour') + ds_9.s_1.max(dim='hour') ) / 9.0

composite_diurnal_peak_phase = (ds_1.s_1.idxmax(dim='hour') + ds_2.s_1.idxmax(dim='hour') + ds_3.s_1.idxmax(dim='hour')
                               +ds_4.s_1.idxmax(dim='hour') + ds_5.s_1.idxmax(dim='hour') + ds_6.s_1.idxmax(dim='hour')
                               +ds_7.s_1.idxmax(dim='hour') + ds_8.s_1.idxmax(dim='hour') + ds_9.s_1.idxmax(dim='hour') ) / 9.0

In [ ]:
composite_diurnal_peak_amplitude = (ds_1.s_1.max(dim='hour') + ds_2.s_1.max(dim='hour') + ds_3.s_1.max(dim='hour')
                                   +ds_4.s_1.max(dim='hour') + ds_5.s_1.max(dim='hour') + ds_6.s_1.max(dim='hour')
                                   +ds_7.s_1.max(dim='hour') + ds_8.s_1.max(dim='hour') + ds_9.s_1.max(dim='hour') + ds_10.s_1.max(dim='hour')) / 10.0

In [ ]:
composite_diurnal_peak_phase = (ds_1.s_1.idxmax(dim='hour') + ds_2.s_1.idxmax(dim='hour') + ds_3.s_1.idxmax(dim='hour')
                               +ds_4.s_1.idxmax(dim='hour') + ds_5.s_1.idxmax(dim='hour') + ds_6.s_1.idxmax(dim='hour')
                               +ds_7.s_1.idxmax(dim='hour') + ds_8.s_1.idxmax(dim='hour') + ds_9.s_1.idxmax(dim='hour') + ds_10.s_1.idxmax(dim='hour')) / 10.0

In [ ]:
#-- Set figure plot bounds
fig = plt.figure(figsize=(12, 10))

gsFig = plt.GridSpec(1, 1)
gsFig.update(left=0.10, right=0.90, bottom = 0.12, top = 0.92)
axFig_1 = plt.subplot(gsFig[0, 0], projection=ccrs.PlateCarree(180))

gsCB = plt.GridSpec(1, 1)
gsCB.update(left=0.14, right=0.86, bottom = 0.06, top = 0.08)
axCB = plt.subplot(gsCB[0, 0])

#-- Xarray plot
image = composite_diurnal_peak_amplitude.plot(ax=axFig_1,
                                              add_colorbar=False,
                                              vmax=0.5e-7,
                                              transform=ccrs.PlateCarree())

#-- Add coastlines
axFig_1.add_feature(cfeature.COASTLINE, edgecolor='black',linewidth=1, zorder=1)
axFig_1.add_feature(cfeature.BORDERS, edgecolor='black',linewidth=2, zorder=1)
axFig_1.add_feature(cfeature.STATES, edgecolor='black',linewidth=1, zorder=1)

#-- Modify gridlines
gl = axFig_1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
gl.xlabels_top = False ; gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER ; gl.yformatter = LATITUDE_FORMATTER
gl.xlocator = mticker.FixedLocator([-120, -110, -100, -90, -80, -70])
gl.ylocator = mticker.FixedLocator([15,25,35,45,55])
gl.xlabel_style = {'size': 20} ; gl.ylabel_style = {'size': 20}

#--Fix colorbar
cb = plt.colorbar(image, cax=axCB, orientation='horizontal', extend="max")
cb.set_label(label='m/s', size=18)
cb.ax.tick_params(labelsize=16)

#--Labels
axFig_1.set_title("Composite diurnal peak amplitude | LR 1-hourly | JJA 1980-1988", fontsize=24, pad=20)

#--save image
plt.savefig('/ihesp/user/asblack/diurnal_precip/figures/composite_diurnal_peak_amplitude_LR_1-hourly_JJA_1980-1988.pdf')

In [ ]:
#-- Set figure plot bounds
fig = plt.figure(figsize=(12, 10))

gsFig = plt.GridSpec(1, 1)
gsFig.update(left=0.10, right=0.90, bottom = 0.12, top = 0.92)
axFig_1 = plt.subplot(gsFig[0, 0], projection=ccrs.PlateCarree(180))

gsCB = plt.GridSpec(1, 1)
gsCB.update(left=0.14, right=0.86, bottom = 0.06, top = 0.08)
axCB = plt.subplot(gsCB[0, 0])

#-- Xarray plot
image = composite_diurnal_peak_phase.plot(ax=axFig_1,
                               add_colorbar=False,
                               levels=[0,3,6,9,12,15,18,21],
                               colors=['navy','mediumblue','royalblue','lightsteelblue','wheat','orange','darkorange','sienna'],
                               transform=ccrs.PlateCarree())

#-- Add coastlines
axFig_1.add_feature(cfeature.COASTLINE, edgecolor='black',linewidth=1, zorder=1)
axFig_1.add_feature(cfeature.BORDERS, edgecolor='black',linewidth=2, zorder=1)
axFig_1.add_feature(cfeature.STATES, edgecolor='black',linewidth=1, zorder=1)

#-- Modify gridlines
gl = axFig_1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
gl.xlabels_top = False ; gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER ; gl.yformatter = LATITUDE_FORMATTER
gl.xlocator = mticker.FixedLocator([-120, -110, -100, -90, -80, -70])
gl.ylocator = mticker.FixedLocator([15,25,35,45,55])
gl.xlabel_style = {'size': 20} ; gl.ylabel_style = {'size': 20}

#--Fix colorbar
cb = plt.colorbar(image, cax=axCB, orientation='horizontal', extend="max")
cb.set_label(label='UTC', size=18)
cb.ax.tick_params(labelsize=16)

#--Labels
axFig_1.set_title("Composite diurnal peak phase | LR 1-hourly | JJA 1980-1988", fontsize=24, pad=20)

#--save image
plt.savefig('/ihesp/user/asblack/diurnal_precip/figures/composite_diurnal_peak_phase_LR_1-hourly_JJA_1980-1988.pdf')

### Plot JJA s_1 yearly result

In [ ]:
s_1.idxmax(dim='hour').plot()

In [ ]:
s_1 = xr.open_dataset('/ihesp/user/asblack/diurnal_precip/cam_h5_HR_PRECT_s1_1987_JJA.nc').s_1

In [ ]:
pp = s_1.idxmax(dim='hour')

#-- Set figure plot bounds
fig = plt.figure(figsize=(12, 10))

gsFig = plt.GridSpec(1, 1)
gsFig.update(left=0.10, right=0.90, bottom = 0.12, top = 0.92)
axFig_1 = plt.subplot(gsFig[0, 0], projection=ccrs.PlateCarree(180))

gsCB = plt.GridSpec(1, 1)
gsCB.update(left=0.14, right=0.86, bottom = 0.06, top = 0.08)
axCB = plt.subplot(gsCB[0, 0])

#-- Xarray plot
image = pp.plot(ax=axFig_1,
                               add_colorbar=False,
                               levels=[0,3,6,9,12,15,18,21],
                               colors=['navy','mediumblue','royalblue','lightsteelblue','wheat','orange','darkorange','sienna'],
                               transform=ccrs.PlateCarree())

#-- Add coastlines
axFig_1.add_feature(cfeature.COASTLINE, edgecolor='black',linewidth=1, zorder=1)
axFig_1.add_feature(cfeature.BORDERS, edgecolor='black',linewidth=2, zorder=1)
axFig_1.add_feature(cfeature.STATES, edgecolor='black',linewidth=1, zorder=1)


#-- Modify gridlines
gl = axFig_1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
gl.xlabels_top = False ; gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER ; gl.yformatter = LATITUDE_FORMATTER
gl.xlocator = mticker.FixedLocator([-120, -110, -100, -90, -80, -70])
gl.ylocator = mticker.FixedLocator([15,25,35,45,55])
gl.xlabel_style = {'size': 20} ; gl.ylabel_style = {'size': 20}

#--Fix colorbar
cb = plt.colorbar(image, cax=axCB, orientation='horizontal', extend="max")
cb.set_label(label='UTC', size=18)
cb.ax.tick_params(labelsize=16)

#--Labels
axFig_1.set_title("Diurnal peak phase | HR 1-hourly | JJA 1987", fontsize=24, pad=20)

#--save image
plt.savefig('/ihesp/user/asblack/diurnal_precip/figures/diurnal_peak_phase_HR_1-hourly_JJA_1987.pdf')

In [ ]:
pp = s_1.max(dim='hour')

#-- Set figure plot bounds
fig = plt.figure(figsize=(12, 10))

gsFig = plt.GridSpec(1, 1)
gsFig.update(left=0.10, right=0.90, bottom = 0.12, top = 0.92)
axFig_1 = plt.subplot(gsFig[0, 0], projection=ccrs.PlateCarree(180))

gsCB = plt.GridSpec(1, 1)
gsCB.update(left=0.14, right=0.86, bottom = 0.06, top = 0.08)
axCB = plt.subplot(gsCB[0, 0])

#-- Xarray plot
image = pp.plot(ax=axFig_1,
                                              add_colorbar=False,
                                              vmax=1e-7,
                                              transform=ccrs.PlateCarree())

#-- Add coastlines
axFig_1.add_feature(cfeature.COASTLINE, edgecolor='black',linewidth=1, zorder=1)
axFig_1.add_feature(cfeature.BORDERS, edgecolor='black',linewidth=2, zorder=1)
axFig_1.add_feature(cfeature.STATES, edgecolor='black',linewidth=1, zorder=1)


#-- Modify gridlines
gl = axFig_1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
gl.xlabels_top = False ; gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER ; gl.yformatter = LATITUDE_FORMATTER
gl.xlocator = mticker.FixedLocator([-120, -110, -100, -90, -80, -70])
gl.ylocator = mticker.FixedLocator([15,25,35,45,55])
gl.xlabel_style = {'size': 20} ; gl.ylabel_style = {'size': 20}

#--Fix colorbar
cb = plt.colorbar(image, cax=axCB, orientation='horizontal', extend="max")
cb.set_label(label='m/s', size=18)
cb.ax.tick_params(labelsize=16)

#--Labels
axFig_1.set_title("Diurnal peak amplitude | HR 1-hourly | JJA 1987", fontsize=24, pad=20)

#--save image
plt.savefig('/ihesp/user/asblack/diurnal_precip/figures/diurnal_peak_amplitude_HR_1-hourly_JJA_1987.pdf')